# Interactive try on CycleGAN

> Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [1]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2340, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2340 (delta 0), reused 2 (delta 0), pack-reused 2337
Receiving objects: 100% (2340/2340), 8.10 MiB | 34.26 MiB/s, done.
Resolving deltas: 100% (1499/1499), done.


In [2]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 676 kB 35.7 MB/s 
     |████████████████████████████████| 68 kB 8.5 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655249 sha256=09160375a511bcee501edf5fc5b1c1b909408d4697c5c867d570bf46d9a4551d
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=880a3b0cb7a8f3a7a375d634c33801f571db2af5234e532dc54b961bafaaf49e
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [4]:
# !bash ./scripts/download_cyclegan_model.sh horse2zebra

# Visualize

In [5]:
from PIL import Image
from io import BytesIO
from pathlib import Path
from ipywidgets import interact, interact_manual, FileUpload, HBox, Output, Button
import sys

In [6]:
sys.args=[]

In [7]:
models_names = ["summer2winter_yosemite","apple2orange", "orange2apple", 
           "winter2summer_yosemite", "horse2zebra", "zebra2horse",
           "monet2photo", "style_monet", "style_cezanne", 
           "style_ukiyoe", "style_vangogh", "sat2map", "map2sat", 
           "cityscapes_photo2label", "cityscapes_label2photo", 
           "facades_photo2label", "facades_label2photo", "iphone2dslr_flower"]

In [8]:
def get_image(uploader):
    images = []
    for name, file_info in uploader.value.items():
        img = Image.open(BytesIO(file_info['content']))
        images.append(img)
    return images

def save_image(img, fname="image1"):
    img.save(f"images/{fname}.jpg")

In [9]:
def generate_pred(uploader, model_name,):
    imgs = get_image(uploader)
    os.system("rm -f images/*")
    result_path = Path(f"./results/{model_name}_pretrained/test_latest/images")
    os.system(f"rm -f {result_path}*")
    for idx, img in enumerate(imgs):
        display(img.resize((500,500)))
        save_image(img, f"image_{idx}")
    print(f"predicting with [{model_name}], pls wait...")
    os.system(f"python test.py --dataroot images --name {model_name}_pretrained --model test --no_dropout")
    
    for img_path in list(result_path.iterdir()):
        if "fake" in img_path.name:
            with Image.open(img_path) as img:
                display(img)

In [10]:
def create_btn(uploader, model_name):
    def click_event(o):
        generate_pred(uploader, model_name)
    btn = Button(description="Transform")
    btn.on_click(click_event)
    return btn

In [11]:
!mkdir -p images

In [12]:
out = Output()
interact_manual2 = interact_manual.options(manual_name="use model")
@interact_manual2
def select_model(model_name = models_names):
    out.clear_output()
    model_path= "/content/pytorch-CycleGAN-and-pix2pix/checkpoints/"\
    +f"{model_name}_pretrained/latest_net_G.pth"
    if Path(model_path).exists()==False:
        print(f"we don't have '{model_name}' yet, download model now...")
        os.system(f"bash ./scripts/download_cyclegan_model.sh {model_name}")
        os.system(f"mkdir -p ./results/{model_name}_pretrained/test_latest/images")
    uploader = FileUpload()

    btn = create_btn(uploader,model_name)
    display(HBox([uploader,btn]))

interactive(children=(Dropdown(description='model_name', options=('summer2winter_yosemite', 'apple2orange', 'o…